# Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams

This notebook implements the methods described in _Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams_, to be presented at the [Computational Legal Studies Conference](https://cclaw.smu.edu.sg/events/computational-legal-studies-2022), March 2022, hosted by the Center for Computational Law at Singapore Management University.

Paper Summary

> This paper presents a novel method for unsupervised machine scoring of short answer and essay question responses, relying solely on a sufficiently large set of responses to a common prompt, absent the need for pre-labeled sample answers—given said prompt is of a particular character. That is, for questions where “good” answers look similar, “wrong” answers are likely to be “wrong” in different ways. Consequently, when a collection of text embeddings for responses to a common prompt are placed in an appropriate feature space, the centroid of their placements can stand in for a model answer, providing a lodestar against which to measure individual responses. This paper examines the efficacy of this method and discusses potential applications.
>
>Current methods for the automated scoring of short answer and essay questions are poorly suited to spontaneous and idiosyncratic assessments. That is, the time saved in grading must be balanced against the time required for the training of a model. This includes tasks such as the creation of pre-labeled sample answers. This limits the utility of machine grading for single classes working with novel assessments. The method described here eliminates the need for the preparation of pre-labeled sample answers. It is the author’s hope that such a method may be leveraged to reduce the time needed to grade free response questions, promoting the increased adoption of formative assessment esp. in contexts like law school instruction which traditionally have relied almost exclusively on summative assessments.
>
>Ranking by the algorithm is found to be statistically significant when compared to a pseudo-random shuffle. To determine how similar a list’s order was to that produced by a human grader, the lowest number of neighbor swaps needed to transform the ordering of these lists into that of the human ordering was calculated. For a dataset including more than one thousand student answers to a set of thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors, the p-value for a paired t-test of the two populations’ swaps, with the pseudo-random group acting as the untreated group and the machine-grader acting as the treatment, came to 0.00000100, allowing us to reject the null hypothesis that the machine's ordering is equivalent to a random shuffle. Additionally, the Cohen’s d for the number of swaps between the pseudo-random ordering and machine ordering was found to be large (i.e., 0.988).

This notebook was used to obtain the results referenced above. Student exam answers and their associated grade data were acquired and this research conducted after approval was granted by Suffolk University’s Office of Research and Sponsored Programs (ORSP) which oversees all human subject research at Suffolk University.

This notebook implements the primary steps of the method described in the paper, namely:  

1. Produce an embedding for each answer that captures as much of the relevant information as possible.
2. Find the centroid for all of the embeddings in your population of answers and impute the location of a  “correct” model answer.
3. Measure the distance between each answer’s embedding and the “correct” answer (e.g., the answers’ centroid or medoid).
4. Convert the answers’ distances from the model answer into z-scores for the population of answers.
5. Translate these z-scores into some known grading scale.
6. Order the answers according to this scale.
7. Compare these orderings to random orderings by seeing how many times you have to change their rankings to obtain the same ordering as one would get if they were ordered by their human-assigned grades.

_Note: The methods described here are the subject of USPTO Patent Application Serial No. 17/246,563._

## Contents

_These, and other internal links (e.g., "back to contents"), may not work if you are viewing the preview of this notebook on GitHub.com. Links to other notebooks, external sites, and exam questions should, however, be okay._

This notebook has three sections:

- [Data: Student Exam Answers](#Data:-Student-Exam-Answers)
- [Code:Code: Libraries and Novel Functions](#Code:-Libraries-and-Novel-Functions)
- [Results](#Results)


## Data: Student Exam Answers

More than one thousand exam answers were obtained as PDF files containing answers to thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors. Each PDF corresponded to a single student exam. These PDFs were parsed to extract their answers, their contents converted into XML of the following format.

```
<EXAM>
    <STUDENT id='ID'>00000000</STUDENT>
    <QUESTION id='Q1'>
        <![CDATA[
            text of written answer to question one
        ]]>
    </QUESTION>
    <QUESTION id='Q2'>
        <![CDATA[
            text of written answer to question two
        ]]>
    </QUESTION>
    <QUESTION id='Q3'>
        <![CDATA[
            text of written answer to question three
        ]]>
    </QUESTION>
</EXAM>
```

These translated files were reviewed by hand and reformatted as needed to correct any formatting errors. Each XML file was then read into a csv file for its associated exam as a single row with their columns corresponding to each of its elements (e.g., ID, Q1, Q2). Additionally, a column stating the number of words contained in each question was appended to the csv file (e.g., size_Q1, size_Q2). E.g., 

|ID|Q1|Q2|size_Q1|size_Q2|
|--|--|--|-------|-------|
|00001|text of 1's ans to q1|text of 1's ans to q2|6|6|
|00002|text of 2's ans to q1|text of 2's ans to q2|6|6|

Instructors also provided scores for each exam question. These were placed in a csv for each exam with the scores on each question associated to the exam ID. E.g., 

|ID|Q1|Q2|
|--|--|--|
|00001|96|93|
|00002|83|89|

A PDF did not extist for every exam as some exams were hand-written. Also, for the _Property Instructor A_ exam, the IDs of two PDFs did not macth any of the IDs provided in the score sheet. To address such issues, you will see that we merge the two data tables above using the default behavior of Panda's merge method (i.e., we kept only the intersection of ID, those found in both files). Consequently, we did not score the exams for which no PDF existed or those for which we could not find a score on the intructor-provided list of scores.

To see the code used to assist in this processing see the following notebook: [Prep Exams](Prep%20Exams.ipynb).

In keeping with the wishes of those instructors who provided exams, only three of the exams are available here for review (i.e., [property_instructor_A](https://colarusso.github.io/free-response-scoring/exam_questions/property_instructor_A.docx), [property_instructor_B](https://colarusso.github.io/free-response-scoring/exam_questions/property_instructor_B.docx), and [crim_instructor_E](https://colarusso.github.io/free-response-scoring/exam_questions/crim_instructor_E.docx)). Another two are on file with the author and may be shared upon request and the assent of their authors. In keeping with the author's wishes, the remaining exam will not be shared. Subject to constraints imposed by the Family Educational Rights and Privacy Act (FERPA), three of the answer sets may be shared upon request. One of these is linked to an exam requiering instructor assent to be shared.

Please note that the exam questiones shared at the links above are docx files and have been redacted to exclude the instructor's name and text that does not include the scored question prompts (e.g., multiple choice questions). 

To facilitate retrieval of the exam answers which were stored in various folders, a list of dictionaries is defined. Each dictionary defines the folder name where csv files can be found as well as the names of its relevant columns for later consideration (i.e., their ID and those questions to be scored). To avoid sharing of this data publicly, as seen below, these files are not included in this repository and were located outside of this repository's folder (i.e., `../data/`). 

In [1]:
exams = [
            {"folder":"../data/property_instructor_A","columns":["ID","SHORT_ANS","Q1","Q2"]},
            {"folder":"../data/property_instructor_B","columns":["ID","Q1","Q2"]},
            {"folder":"../data/environ_instructor_B","columns":["ID","Q2"]},
            {"folder":"../data/PR_instructor_C","columns":["ID","Q1","Q2"]},
            {"folder":"../data/contracts_instructor_D","columns":["ID","Q1","Q2","Q3"]},
            {"folder":"../data/crim_instructor_E","columns":["ID","Q1","Q2"]}
          ]

[back to contents](#Contents)

## Code: Libraries and Novel Functions

### Python Libraries et al.

First we'll load the following libraries. These are needed to perform our scoring. 

In [2]:
import csv
import pandas as pd
import numpy as np
from numpy import var
from math import sqrt
import spacy
from scipy import stats
from sklearn.preprocessing import normalize

# https://spacy.io/models/en#en_core_web_lg
nlp1 = spacy.load('en_core_web_lg') 
# https://github.com/explosion/spacy-transformers/tree/88814f5f4be7f0d4c784d8500c558d9ba06b9a56
nlp2 = spacy.load("en_trf_distilbertbaseuncased_lg") 
nlp3 = spacy.load("en_trf_robertabase_lg") 

In [3]:
def score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=1):

    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    M = []  
    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    R = []

    # for every exam in the list of exams  
    for exam in exams:

        print("\n=========================================================")
        print(exam["folder"])
        print("=========================================================\n")

        # load this exam's answer texts
        texts = pd.read_csv("%s/texts.csv"%exam["folder"], encoding="utf-8")
        # load this exam's grades
        actual = pd.read_csv("%s/actual.csv"%exam["folder"], encoding="utf-8")

        # print the relevant columns for this exam (i.e., ID and questions to score)
        print(exam["columns"])

        # score the questions listed above
        output_df = grade(texts[exam["columns"]],model,normv,score,goal)
        
        # below we want the average word counts for questions
        # so we need a df with only the questions actually used here
        # i.e., the inner merge. output_df doesn't have the texts columns
        # so we'll do a second merge here real quick to facilitate that count
        texts_df = actual.merge(texts, on="ID")

        # only include grades for those questions defined above
        actual = actual[[x for x in actual.columns if x in exam["columns"]]]

        # merge the machine-scored dataframe with that of the actual grades
        df_exam = actual.merge(output_df, on="ID")
        
        # display a preview of the two scores (first few rows of the merged dataframe)
        # I'm excluding the ID here to avoid sharing IDs in the cell's output
        display(df_exam[[x for x in list(df_exam.columns.values) if (x not in ["ID"])]].head())

        print("Number of entries:",len(df_exam))

        # for each of the questions that were scored
        for qn in [x for x in exam["columns"] if (x not in ["ID"])]:
            mean_words = int(round(texts_df["size_%s"%qn].mean()))
            print("\n%s\nMean Words:"%qn,mean_words)

            # create a dataframe with the grade and machine scores for this question
            df = df_exam[["ID", "%s_x"%qn, "%s_y"%qn]]

            # ideally the granularity of the machine and human scoring should be the same
            # to avoid false precision. It may be worth imposing this in some manner 
            # simmilar to this comment. 
            # df["%s_y"%qn] = round(df["%s_x"%qn].max()*(df["%s_y"%qn]/100))

            # sort answers by the machine scores
            df = df.sort_values(by='%s_y'%qn, ascending=True)

            # create an list of these scores then count the swaps needed to bring them into 
            # agreement with the grades. Append this to our list of swaps needed to move from 
            # machine-scored ordering to that of the graded exams
            arr_ = df[df.columns[1]].values.copy()
            n = len(arr_)
            M_o = countSwaps(arr_, n)
            M.append(M_o)

            # we're going to do the same as the above but for pseudo-random shuffles.
            # we'll shuffle and count swaps for the number of times defined in `runs`
            # and average the results 
            tmp = []
            for x in range(runs):
                # note the use of `sample.` This is where we get our pseudo-random ordering
                arr_ = df[df.columns[1]].sample(frac=1).values.copy()
                n = len(arr_)
                R_o = countSwaps(arr_, n)
                tmp.append(R_o)

            R_o = np.array(tmp).mean()
            R.append(R_o)

            print("Algo score, swaps needed:",M_o)
            print("Swaps needed for first 10 out of %s pseudo-random runs:"%runs)
            print(tmp[:10])
            print("Pseudo-random, avergae swaps needed:",R_o)

    print("\n=========================================================")
    print("Swaps (Machine):")
    print(M)
    print("N:",len(M),"\tMean: ",np.mean(M),"\tVar: ",np.var(M))
    print("Avergae swaps (Pseudo-random):")
    print(R)
    print("N:",len(R),"\tMean: ",np.mean(R),"\tVar: ",np.var(R),"\n")

    # Print out a summary table of the percent difference between pseudo-random and machine swaps
    pdiff = []
    i = 0
    for x in R:
        if x != 0 and M[i] != 0:
            pdiff_ = 100*(x-M[i])/((x+M[i])/2)
            pdiff.append(pdiff_)
            print( i+1,")",x,">",M[i],"=>",pdiff_ )
        else:
            print("error")
        i = i + 1

    print("\nAvergae percent difference between swaps needed for pseudo-random and machine sorting:")
    print(np.array(pdiff).mean(),"\n")
        
    # run t test for the two populations of swaps
    sample_n = len(M)
    print(stats.ttest_rel(M[:sample_n], R[:sample_n]))
    # return the effect size
    print("Cohen's d: ",cohend(R[:sample_n], M[:sample_n]))

In [4]:
def grade(df,model,normv,score=0,goal="centroid"):
       
    df = df.copy()    
    
    # exclude the ID column to get only the answer texts
    questions = [x for x in list(df.columns.values) if (x not in ["ID"])]
    # vectorize the text of each answer
    for x in questions:
        df["%s_vec"%x] = df[x].apply(vectorize, args=(model,normv)) 
    
    # for each question
    for x in questions:
        
        # based on the `goal` parameter grade, against either the centroid or medoid
        # that is, set either as the model answer aginst which to grade others
        if goal=="centroid":
            # centroid 
            best_ans = df["%s_vec"%x].mean()
        else:
            # medoid 
            best_ans = df["%s_vec"%x].iloc[best_answer(df,x)]

        # measure each answer's distance from the model answer above
        df["%s_dist"%x] = df["%s_vec"%x].apply(distance, args=(best_ans,))
        
        # replace the text of the answer with its z-score
        df[x] =  df["%s_vec"%x].apply(z_score, args=(df,x,best_ans))
        
        # translate the z-score into a standard grade using
        # the translation defined by `score`
        if score==0:
            df[x] =  df[x].apply(score_0)
        elif score==1:
            df[x] =  df[x].apply(score_1)
            
    # create output df without the vectors or distance measures
    output_df = df[[ x for x in list(df.columns.values) if ("_vec" not in x) and ("_dist" not in x) ]].copy()
            
    return output_df

In [5]:
def vectorize(row,model='nlp1',normv=0): 

    # use a method for vectorization based on the `model` parameter
    # note: some methods can work across the entire text of a qustion
    # others requier vectorization at the sentence level
    if model=='nlp1':
        nlp = nlp1
        mode = 0
    elif model=='nlp2':
        nlp = nlp2
        mode = 1
    elif model=='nlp3':
        nlp = nlp3
        mode = 1
        
    try:
        if mode == 1:
            # vectorize at the sentence level
            doc = nlp(row)
            tmp_df = pd.DataFrame({'sent' : []})
            i=0 
            for sent in doc.sents:
                sent_vec = nlp(sent.string.strip()).vector
                # if normv=1 normalize each sentence vector
                if normv==1:
                    sent_vec = norm(sent_vec)
                if i==0:
                    tmp_df = pd.DataFrame([[sent_vec]],columns=["sent"])
                else:
                    tmp_df = tmp_df.append(pd.DataFrame([[sent_vec]],columns=["sent"]), ignore_index=True)
                i+=1
            # average sentence vectors 
            output = tmp_df["sent"].mean()
        else:
            # vectorize all input text together
            output = nlp(row).vector

        # if normv=1 normalize the answer's vector
        if normv==1:
            output = norm(output)  
    
        return output
    except:
        return np.NaN
    
def norm(row):
    # normalize vector
    matrix = row.reshape(1,-1).astype(np.float64)
    return normalize(matrix, axis=1, norm='l1')[0]

In [6]:
def best_answer(df,column):
    # Return the medoid (like the centroid, but confined to a member of the group) 
    # i.e., the best answer in the group of answers.
    tmp_df = df[[ "%s_vec"%column ]].copy()
    centroid = tmp_df["%s_vec"%column].mean()
    tmp_df["%s_dist"%column] = tmp_df["%s_vec"%column].apply(distance, args=(centroid,))
    tmp_df = tmp_df[tmp_df["%s_dist"%column]==tmp_df["%s_dist"%column].min()]
    return tmp_df.index[0]

def distance(row,centroid):
    # calculate the Euclidean distance between centroid and row
    # See https://stackoverflow.com/a/1401828
    return np.linalg.norm(centroid-row)

In [7]:
def z_score(row,df,x,best):
    # calculate the z-score for a given distance
    return -1*(np.linalg.norm(best-row)-df["%s_dist"%x].mean())/df["%s_dist"%x].std()

In [8]:
# Define grade bin boarders based on z-score
# --------------------------------------
# There's no standard mapping between z-scores and letter grades
# However, it's common for schools to set their own curve which 
# is in the same spirit. Therefor I have created a mapping where
# the average answer recieves a B. Others might choose differently

grade_bins = [
              2,    # A+
              1.5,  # A
              1,    # A-
              0.5,  # B+
              0,    # B
              -0.5, # B-
              -1,   # C+
              -1.5, # C
              -2,   # C-
              -2.5, # D+
              -3,   # D
              -3.5  # D-
             ]

# Assign numeric grades based on z-score
# -----------------------------------------
# This was the original z-score to percentage grade translation.
# I took the above and translated it into numberic grades
# with the scale bottoming out at 59 and topping out at 97
def score_0(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93+((row-grade_bins[1])/(grade_bins[0]-grade_bins[1]))*4
    elif row >= grade_bins[2]:
        grade = 90+((row-grade_bins[2])/(grade_bins[1]-grade_bins[2]))*3
    elif row >= grade_bins[3]:
        grade = 87+((row-grade_bins[3])/(grade_bins[2]-grade_bins[3]))*3
    elif row >= grade_bins[4]:
        grade = 83+((row-grade_bins[4])/(grade_bins[3]-grade_bins[4]))*4
    elif row >= grade_bins[5]:
        grade = 80+((row-grade_bins[5])/(grade_bins[4]-grade_bins[5]))*3
    elif row >= grade_bins[6]:
        grade = 77+((row-grade_bins[6])/(grade_bins[5]-grade_bins[6]))*3
    elif row >= grade_bins[7]:
        grade = 73+((row-grade_bins[7])/(grade_bins[6]-grade_bins[7]))*4
    elif row >= grade_bins[8]:
        grade = 70+((row-grade_bins[8])/(grade_bins[7]-grade_bins[8]))*3
    elif row >= grade_bins[9]:
        grade = 67+((row-grade_bins[9])/(grade_bins[8]-grade_bins[9]))*3
    elif row >= grade_bins[10]:
        grade = 63+((row-grade_bins[10])/(grade_bins[9]-grade_bins[10]))*4
    elif row >= grade_bins[11]:
        grade = 60+(row-grade_bins[11])*(3/(grade_bins[10]-grade_bins[11]))
    else:
        grade = 59
        
    # The final output is rounded to avoid false percision. A better 
    # approach is to bin all of the scores. This is done in score_1 below
    return round(grade)

# Assign numeric grades based on z-score
# -----------------------------------------
# Upon subsequent reflection, I realized that binned scores were preferable
# to a continuous score with dropoffs at 59 and 97. The above translation,
# however, was used on the first run of this code and so is included here
# for compleatness.  
def score_1(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93
    elif row >= grade_bins[2]:
        grade = 90
    elif row >= grade_bins[3]:
        grade = 87
    elif row >= grade_bins[4]:
        grade = 83
    elif row >= grade_bins[5]:
        grade = 80
    elif row >= grade_bins[6]:
        grade = 77
    elif row >= grade_bins[7]:
        grade = 73
    elif row >= grade_bins[8]:
        grade = 70
    elif row >= grade_bins[9]:
        grade = 67
    elif row >= grade_bins[10]:
        grade = 63
    elif row >= grade_bins[11]:
        grade = 60
    else:
        grade = 59
    return grade

In [9]:
# The following functions are used to count the number of adjacent swaps needed
# to change one ordering into another. It was written by Shivam Gupta
# See https://www.geeksforgeeks.org/number-swaps-sort-adjacent-swapping-allowed/
# -----------------------------------------------------
# python 3 program to count number of swaps required 
# to sort an array when only swapping of adjacent 
# elements is allowed. 
# include <bits/stdc++.h> 

#This function merges two sorted arrays and returns inversion count in the arrays.*/ 
def merge(arr, temp, left, mid, right): 
    inv_count = 0

    i = left #i is index for left subarray*/ 
    j = mid #i is index for right subarray*/ 
    k = left #i is index for resultant merged subarray*/ 
    while ((i <= mid - 1) and (j <= right)): 
        if (arr[i] <= arr[j]): 
            temp[k] = arr[i] 
            k += 1
            i += 1
        else: 
            temp[k] = arr[j] 
            k += 1
            j += 1

            #this is tricky -- see above explanation/ 
            # diagram for merge()*/ 
            inv_count = inv_count + (mid - i) 

    #Copy the remaining elements of left subarray 
    # (if there are any) to temp*/ 
    while (i <= mid - 1): 
        temp[k] = arr[i] 
        k += 1
        i += 1

    #Copy the remaining elements of right subarray 
    # (if there are any) to temp*/ 
    while (j <= right): 
        temp[k] = arr[j] 
        k += 1
        j += 1

    # Copy back the merged elements to original array*/ 
    for i in range(left,right+1,1): 
        arr[i] = temp[i] 

    return inv_count 

#An auxiliary recursive function that sorts the input 
# array and returns the number of inversions in the 
# array. */ 
def _mergeSort(arr, temp, left, right): 
    inv_count = 0
    if (right > left): 
        # Divide the array into two parts and call 
        #_mergeSortAndCountInv() 
        # for each of the parts */ 
        mid = int((right + left)/2) 

        #Inversion count will be sum of inversions in 
        # left-part, right-part and number of inversions 
        # in merging */ 
        inv_count = _mergeSort(arr, temp, left, mid) 
        inv_count += _mergeSort(arr, temp, mid+1, right) 

        # Merge the two parts*/ 
        inv_count += merge(arr, temp, left, mid+1, right) 

    return inv_count 

#This function sorts the input array and returns the 
#number of inversions in the array */ 
def countSwaps(arr, n): 
    temp = [0 for i in range(n)] 
    return _mergeSort(arr, temp, 0, n - 1) 

In [10]:
# The following function calculates Cohen d (the effect size)
# based on two populations. It was written by Jason Brownlee
# See https://machinelearningmastery.com/effect-size-measures-in-python/
# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

[back to contents](#Contents)

# Results

The arrangement of parameters below corresponds to those used when all of the exam data was first run through the above scoring method. To avoid biasing parameter selection, prior work on the method did NOT use the exam data referenced. You will note that the count of swapps needed for the pseudo-random shuffle differs from that in Table 1. That is because the output here represents a subsequent run of the method. To obtain the values found in table 2, the pseudo-random shuffle was repeated 100,000 times, and the number of swaps were averaged. These results can be found in the cell after the one below. 

In [11]:
score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=1)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,86,83,87
1,10,30,32,88,86,87
2,12,39,24,90,90,85
3,15,38,16,88,87,86
4,13,43,19,91,89,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 996
Swaps needed for first 10 out of 1 pseudo-random runs:
[1398]
Pseudo-random, avergae swaps needed: 1398.0

Q1
Mean Words: 2048
Algo score, swaps needed: 1017
Swaps needed for first 10 out of 1 pseudo-random runs:
[1863]
Pseudo-random, avergae swaps needed: 1863.0

Q2
Mean Words: 947
Algo score, swaps needed: 828
Swaps needed for first 10 out of 1 pseudo-random runs:
[1633]
Pseudo-random, avergae swaps needed: 1633.0

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,88,86
1,20.0,18.0,80,84
2,24.0,25.0,89,80
3,18.0,22.0,83,91
4,17.0,17.0,78,79


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1290
Swaps needed for first 10 out of 1 pseudo-random runs:
[1810]
Pseudo-random, avergae swaps needed: 1810.0

Q2
Mean Words: 1546
Algo score, swaps needed: 1326
Swaps needed for first 10 out of 1 pseudo-random runs:
[1779]
Pseudo-random, avergae swaps needed: 1779.0

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 911
Swaps needed for first 10 out of 1 pseudo-random runs:
[1136]
Pseudo-random, avergae swaps needed: 1136.0

Q2
Mean Words: 570
Algo score, swaps needed: 795
Swaps needed for first 10 out of 1 pseudo-random runs:
[1268]
Pseudo-random, avergae swaps needed: 1268.0

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,87,81
1,53,27,27,82,80,89
2,38,36,20,81,84,81
3,66,37,37,83,88,87
4,71,34,38,85,88,85


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1178
Swaps needed for first 10 out of 1 pseudo-random runs:
[1473]
Pseudo-random, avergae swaps needed: 1473.0

Q2
Mean Words: 1050
Algo score, swaps needed: 993
Swaps needed for first 10 out of 1 pseudo-random runs:
[1322]
Pseudo-random, avergae swaps needed: 1322.0

Q3
Mean Words: 839
Algo score, swaps needed: 807
Swaps needed for first 10 out of 1 pseudo-random runs:
[1462]
Pseudo-random, avergae swaps needed: 1462.0

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,89,89
1,24.0,25.0,82,85
2,26.5,24.0,63,75
3,23.5,23.0,87,87
4,26.5,21.0,87,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1792
Swaps needed for first 10 out of 1 pseudo-random runs:
[2206]
Pseudo-random, avergae swaps needed: 2206.0

Q2
Mean Words: 2137
Algo score, swaps needed: 1637
Swaps needed for first 10 out of 1 pseudo-random runs:
[2265]
Pseudo-random, avergae swaps needed: 2265.0

Swaps (Machine):
[996, 1017, 828, 1290, 1326, 115, 911, 795, 1178, 993, 807, 1792, 1637]
N: 13 	Mean:  1052.6923076923076 	Var:  163027.44378698224
Avergae swaps (Pseudo-random):
[1398.0, 1863.0, 1633.0, 1810.0, 1779.0, 135.0, 1136.0, 1268.0, 1473.0, 1322.0, 1462.0, 2206.0, 2265.0]
N: 13 	Mean:  1519.2307692307693 	Var:  268927.56213017757 

1 ) 1398.0 > 996 => 33.583959899749374
2 ) 1863.0 > 1017 => 58.75
3 ) 1633.0 > 828 => 65.42056074766356
4 ) 1810.0 > 1290 => 33.54838709677419
5 ) 1779.0 > 1326 => 29.17874396135266
6 ) 135.0 > 115 => 16.0
7 ) 1136.0 > 911 => 21.983390327308257
8 ) 1268.0 > 795 => 45.85555016965584
9 ) 1473.0 > 1178 => 22.2557525462

As stated above, the following cell is the same as the above except the pseudo-random shuffle was run 100,000 times and the swaps needed averaged. 

In [12]:
score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,86,83,87
1,10,30,32,88,86,87
2,12,39,24,90,90,85
3,15,38,16,88,87,86
4,13,43,19,91,89,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 996
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1409, 1410, 1348, 1172, 1611, 1641, 1526, 1315, 1632, 1452]
Pseudo-random, avergae swaps needed: 1499.59968

Q1
Mean Words: 2048
Algo score, swaps needed: 1017
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1522, 1568, 1469, 1388, 1415, 1403, 1526, 1482, 1563, 1582]
Pseudo-random, avergae swaps needed: 1566.16943

Q2
Mean Words: 947
Algo score, swaps needed: 828
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1421, 1604, 1442, 1597, 1592, 1287, 1615, 1587, 1654, 1566]
Pseudo-random, avergae swaps needed: 1560.34999

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,88,86
1,20.0,18.0,80,84
2,24.0,25.0,89,80
3,18.0,22.0,83,91
4,17.0,17.0,78,79


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1290
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1942, 1633, 1789, 1564, 1916, 1859, 1956, 1829, 1741, 1871]
Pseudo-random, avergae swaps needed: 1838.26881

Q2
Mean Words: 1546
Algo score, swaps needed: 1326
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1771, 1767, 1871, 1714, 1773, 1750, 1857, 1898, 1583, 1661]
Pseudo-random, avergae swaps needed: 1757.00175

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 911
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1319, 1391, 1273, 1277, 1275, 1524, 1246, 1275, 1396, 1426]
Pseudo-random, avergae swaps needed: 1324.12667

Q2
Mean Words: 570
Algo score, swaps needed: 795
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1219, 1082, 1346, 1338, 1221, 1254, 1203, 1161, 1216, 1505]
Pseudo-random, avergae swaps needed: 1219.62178

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,87,81
1,53,27,27,82,80,89
2,38,36,20,81,84,81
3,66,37,37,83,88,87
4,71,34,38,85,88,85


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1178
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1683, 1557, 1429, 1301, 1244, 1278, 1350, 1505, 1487, 1438]
Pseudo-random, avergae swaps needed: 1472.05595

Q2
Mean Words: 1050
Algo score, swaps needed: 993
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1540, 1582, 1398, 1343, 1385, 1507, 1607, 1414, 1518, 1710]
Pseudo-random, avergae swaps needed: 1450.13959

Q3
Mean Words: 839
Algo score, swaps needed: 807
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1360, 1469, 1447, 1557, 1203, 1418, 1625, 1477, 1374, 1487]
Pseudo-random, avergae swaps needed: 1458.96619

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,89,89
1,24.0,25.0,82,85
2,26.5,24.0,63,75
3,23.5,23.0,87,87
4,26.5,21.0,87,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1792
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1840, 1761, 1949, 2107, 2039, 2040, 2444, 2128, 2043, 2200]
Pseudo-random, avergae swaps needed: 2045.01444

Q2
Mean Words: 2137
Algo score, swaps needed: 1637
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2036, 1928, 2126, 2065, 1970, 2059, 2055, 2088, 2002, 1827]
Pseudo-random, avergae swaps needed: 1993.8995

Swaps (Machine):
[996, 1017, 828, 1290, 1326, 115, 911, 795, 1178, 993, 807, 1792, 1637]
N: 13 	Mean:  1052.6923076923076 	Var:  163027.44378698224
Avergae swaps (Pseudo-random):
[1499.59968, 1566.16943, 1560.34999, 1838.26881, 1757.00175, 182.54449, 1324.12667, 1219.62178, 1472.05595, 1450.13959, 1458.96619, 2045.01444, 1993.8995]
N: 13 	Mean:  1489.827559230769 	Var:  198331.6689495791 

1 ) 1499.59968 > 996 => 40.35901142606333
2 ) 1566.16943 > 1017 => 42.51904064999716
3 ) 1560.34999 > 828 => 61.32685687326756
4 ) 1838.26881 >

The following cells present a variety of parameters selections, including a mix of vector normalization, grade translations, "best answer" selections, and the number of pseudo-random runs.

In [13]:
score_exams(exams,model='nlp1',normv=1,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,87,85,86
1,10,30,32,87,83,86
2,12,39,24,89,90,85
3,15,38,16,88,86,86
4,13,43,19,92,89,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1005
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1314, 1601, 1493, 1686, 1608, 1466, 1535, 1703, 1661, 1445]
Pseudo-random, avergae swaps needed: 1499.46311

Q1
Mean Words: 2048
Algo score, swaps needed: 1056
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1718, 1358, 1752, 1572, 1748, 1553, 1780, 1340, 1559, 1609]
Pseudo-random, avergae swaps needed: 1566.93115

Q2
Mean Words: 947
Algo score, swaps needed: 963
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1404, 1531, 1285, 1531, 1650, 1609, 1846, 1256, 1428, 1657]
Pseudo-random, avergae swaps needed: 1560.36848

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,85,83
1,20.0,18.0,79,83
2,24.0,25.0,89,79
3,18.0,22.0,83,92
4,17.0,17.0,82,79


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1255
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1861, 1835, 1665, 1905, 1822, 1821, 1659, 1522, 1936, 1996]
Pseudo-random, avergae swaps needed: 1837.32704

Q2
Mean Words: 1546
Algo score, swaps needed: 1215
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1733, 1724, 1587, 1786, 1766, 1574, 2111, 1469, 1604, 1731]
Pseudo-random, avergae swaps needed: 1757.71825

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 921
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1434, 1364, 1411, 1214, 1430, 1450, 1214, 1345, 1302, 1530]
Pseudo-random, avergae swaps needed: 1324.25476

Q2
Mean Words: 570
Algo score, swaps needed: 891
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1272, 1121, 1328, 1259, 1132, 1138, 1433, 1117, 1239, 1270]
Pseudo-random, avergae swaps needed: 1219.49475

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,85,80
1,53,27,27,81,79,89
2,38,36,20,81,83,81
3,66,37,37,82,88,87
4,71,34,38,85,91,83


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1189
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1461, 1395, 1447, 1404, 1560, 1348, 1495, 1372, 1423, 1698]
Pseudo-random, avergae swaps needed: 1472.28807

Q2
Mean Words: 1050
Algo score, swaps needed: 1049
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1390, 1410, 1592, 1465, 1259, 1386, 1318, 1428, 1376, 1746]
Pseudo-random, avergae swaps needed: 1450.09882

Q3
Mean Words: 839
Algo score, swaps needed: 814
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1399, 1341, 1569, 1472, 1495, 1573, 1484, 1610, 1425, 1461]
Pseudo-random, avergae swaps needed: 1457.79426

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,91
1,24.0,25.0,81,84
2,26.5,24.0,61,74
3,23.5,23.0,85,85
4,26.5,21.0,86,91


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1658
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2002, 2088, 2169, 2054, 1894, 2094, 2083, 2064, 2042, 1989]
Pseudo-random, avergae swaps needed: 2046.27619

Q2
Mean Words: 2137
Algo score, swaps needed: 1580
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1969, 1924, 2048, 2011, 2018, 2110, 2081, 1809, 1828, 1960]
Pseudo-random, avergae swaps needed: 1993.64008

Swaps (Machine):
[1005, 1056, 963, 1255, 1215, 113, 921, 891, 1189, 1049, 814, 1658, 1580]
N: 13 	Mean:  1054.5384615384614 	Var:  132621.94082840235
Avergae swaps (Pseudo-random):
[1499.46311, 1566.93115, 1560.36848, 1837.32704, 1757.71825, 182.49928, 1324.25476, 1219.49475, 1472.28807, 1450.09882, 1457.79426, 2046.27619, 1993.64008]
N: 13 	Mean:  1489.8580184615384 	Var:  198423.90854044695 

1 ) 1499.46311 > 1005 => 39.486555663421214
2 ) 1566.93115 > 1056 => 38.95879234191869
3 ) 1560.36848 > 963 => 47.346908288241764
4 ) 183

In [14]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,87,83,83
1,10,30,32,83,83,83
2,12,39,24,87,87,83
3,15,38,16,87,83,83
4,13,43,19,90,87,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 996
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1552, 1407, 1568, 1538, 1445, 1351, 1419, 1339, 1554, 1515]
Pseudo-random, avergae swaps needed: 1499.32791

Q1
Mean Words: 2048
Algo score, swaps needed: 1060
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1712, 1485, 1395, 1465, 1397, 1498, 1523, 1584, 1810, 1779]
Pseudo-random, avergae swaps needed: 1566.24042

Q2
Mean Words: 947
Algo score, swaps needed: 948
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1625, 1373, 1433, 1581, 1434, 1736, 1316, 1371, 1597, 1611]
Pseudo-random, avergae swaps needed: 1559.97103

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,83
1,20.0,18.0,77,80
2,24.0,25.0,87,77
3,18.0,22.0,80,90
4,17.0,17.0,80,77


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1338
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1808, 1845, 1771, 1773, 1960, 1839, 2067, 1957, 1653, 1765]
Pseudo-random, avergae swaps needed: 1837.96365

Q2
Mean Words: 1546
Algo score, swaps needed: 1254
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1873, 1527, 1907, 2028, 1871, 1698, 1878, 1483, 1830, 1600]
Pseudo-random, avergae swaps needed: 1757.58371

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 904
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1308, 1421, 1184, 1259, 1194, 1370, 1125, 1334, 1227, 1144]
Pseudo-random, avergae swaps needed: 1324.44146

Q2
Mean Words: 570
Algo score, swaps needed: 1046
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1193, 1171, 1260, 1205, 1267, 1321, 1224, 1245, 1390, 1376]
Pseudo-random, avergae swaps needed: 1219.90126

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,83,80
1,53,27,27,80,77,87
2,38,36,20,80,80,80
3,66,37,37,80,87,83
4,71,34,38,83,90,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1174
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1579, 1236, 1253, 1596, 1363, 1423, 1438, 1427, 1703, 1540]
Pseudo-random, avergae swaps needed: 1473.19909

Q2
Mean Words: 1050
Algo score, swaps needed: 1084
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1500, 1478, 1017, 1329, 1639, 1421, 1367, 1390, 1531, 1337]
Pseudo-random, avergae swaps needed: 1450.57749

Q3
Mean Words: 839
Algo score, swaps needed: 784
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1301, 1392, 1295, 1662, 1557, 1436, 1347, 1440, 1460, 1537]
Pseudo-random, avergae swaps needed: 1458.00432

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,90
1,24.0,25.0,80,83
2,26.5,24.0,60,73
3,23.5,23.0,83,83
4,26.5,21.0,83,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1627
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1951, 2157, 1752, 2207, 2092, 2256, 2001, 1910, 1939, 1973]
Pseudo-random, avergae swaps needed: 2045.8972

Q2
Mean Words: 2137
Algo score, swaps needed: 1622
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2092, 1693, 1981, 1985, 1861, 1945, 1779, 1799, 2189, 1763]
Pseudo-random, avergae swaps needed: 1993.81891

Swaps (Machine):
[996, 1060, 948, 1338, 1254, 102, 904, 1046, 1174, 1084, 784, 1627, 1622]
N: 13 	Mean:  1072.2307692307693 	Var:  138384.02366863904
Avergae swaps (Pseudo-random):
[1499.32791, 1566.24042, 1559.97103, 1837.96365, 1757.58371, 182.66296, 1324.44146, 1219.90126, 1473.19909, 1450.57749, 1458.00432, 2045.8972, 1993.81891]
N: 13 	Mean:  1489.9684161538462 	Var:  198360.28479390612 

1 ) 1499.32791 > 996 => 40.34162468050141
2 ) 1566.24042 > 1060 => 38.552481040559115
3 ) 1559.97103 > 948 => 48.80208125849045
4 ) 1837.96

In [15]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,83,83,83
1,10,30,32,83,77,83
2,12,39,24,83,87,83
3,15,38,16,87,87,83
4,13,43,19,97,90,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1131
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1363, 1535, 1584, 1505, 1604, 1422, 1685, 1706, 1384, 1418]
Pseudo-random, avergae swaps needed: 1499.20232

Q1
Mean Words: 2048
Algo score, swaps needed: 1092
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1638, 1697, 1500, 1355, 1359, 1616, 1378, 1550, 1275, 1697]
Pseudo-random, avergae swaps needed: 1566.36381

Q2
Mean Words: 947
Algo score, swaps needed: 992
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1543, 1531, 1445, 1542, 1496, 1779, 1732, 1856, 1572, 1512]
Pseudo-random, avergae swaps needed: 1559.38205

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,83
1,20.0,18.0,80,77
2,24.0,25.0,87,80
3,18.0,22.0,80,90
4,17.0,17.0,80,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1292
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1689, 1843, 1882, 1926, 2113, 2113, 1906, 1859, 1856, 1838]
Pseudo-random, avergae swaps needed: 1837.63917

Q2
Mean Words: 1546
Algo score, swaps needed: 1607
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1856, 1377, 1723, 1806, 1702, 1679, 1803, 1793, 1746, 1940]
Pseudo-random, avergae swaps needed: 1757.54437

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 885
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1320, 1504, 1493, 1111, 1355, 1419, 1250, 1100, 1139, 1367]
Pseudo-random, avergae swaps needed: 1324.67714

Q2
Mean Words: 570
Algo score, swaps needed: 1036
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1277, 1241, 1111, 1173, 1121, 1431, 1248, 1224, 1181, 1270]
Pseudo-random, avergae swaps needed: 1219.54373

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,80,77
1,53,27,27,77,80,87
2,38,36,20,77,83,80
3,66,37,37,80,87,83
4,71,34,38,83,90,83


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1283
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1511, 1321, 1539, 1728, 1407, 1275, 1380, 1491, 1279, 1522]
Pseudo-random, avergae swaps needed: 1472.43414

Q2
Mean Words: 1050
Algo score, swaps needed: 1158
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1333, 1507, 1423, 1374, 1313, 1444, 1339, 1399, 1467, 1278]
Pseudo-random, avergae swaps needed: 1450.33686

Q3
Mean Words: 839
Algo score, swaps needed: 927
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1469, 1397, 1342, 1368, 1454, 1473, 1685, 1542, 1620, 1327]
Pseudo-random, avergae swaps needed: 1457.95828

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,83,90
1,24.0,25.0,77,80
2,26.5,24.0,63,70
3,23.5,23.0,80,80
4,26.5,21.0,83,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1846
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1787, 1891, 2253, 2351, 1935, 1705, 2222, 1805, 2005, 2157]
Pseudo-random, avergae swaps needed: 2045.28654

Q2
Mean Words: 2137
Algo score, swaps needed: 1645
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1968, 1923, 1855, 2100, 1990, 1975, 2181, 1828, 2094, 1865]
Pseudo-random, avergae swaps needed: 1994.39233

Swaps (Machine):
[1131, 1092, 992, 1292, 1607, 140, 885, 1036, 1283, 1158, 927, 1846, 1645]
N: 13 	Mean:  1156.4615384615386 	Var:  165954.0946745562
Avergae swaps (Pseudo-random):
[1499.20232, 1566.36381, 1559.38205, 1837.63917, 1757.54437, 182.61545, 1324.67714, 1219.54373, 1472.43414, 1450.33686, 1457.95828, 2045.28654, 1994.39233]
N: 13 	Mean:  1489.7981684615384 	Var:  198350.63073100237 

1 ) 1499.20232 > 1131 => 27.998022600786094
2 ) 1566.36381 > 1092 => 35.68840413908585
3 ) 1559.38205 > 992 => 44.47644757867603
4 ) 1837

In [16]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,87,83,87
1,10,30,32,87,83,87
2,12,39,24,90,80,83
3,15,38,16,90,83,87
4,13,43,19,90,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1003
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1609, 1416, 1607, 1431, 1651, 1475, 1564, 1817, 1637, 1363]
Pseudo-random, avergae swaps needed: 1499.83852

Q1
Mean Words: 2048
Algo score, swaps needed: 1199
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1460, 1456, 1627, 1600, 1844, 1411, 1713, 1627, 1351, 1495]
Pseudo-random, avergae swaps needed: 1566.62592

Q2
Mean Words: 947
Algo score, swaps needed: 907
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1632, 1571, 1562, 1457, 1583, 1691, 1407, 1691, 1537, 1501]
Pseudo-random, avergae swaps needed: 1560.11682

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,87
1,20.0,18.0,70,80
2,24.0,25.0,90,80
3,18.0,22.0,77,87
4,17.0,17.0,77,80


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1293
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2067, 1614, 1863, 1565, 1793, 1948, 1911, 1594, 1858, 1906]
Pseudo-random, avergae swaps needed: 1837.725

Q2
Mean Words: 1546
Algo score, swaps needed: 1274
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1717, 1584, 1762, 1854, 1624, 1733, 1722, 1570, 1919, 1873]
Pseudo-random, avergae swaps needed: 1757.04872

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 932
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1135, 1185, 1335, 1266, 1116, 1315, 1489, 1448, 1439, 1322]
Pseudo-random, avergae swaps needed: 1324.63885

Q2
Mean Words: 570
Algo score, swaps needed: 839
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1304, 1241, 1370, 1270, 1060, 1292, 1270, 1312, 1323, 1173]
Pseudo-random, avergae swaps needed: 1219.81824

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,77,80,80
1,53,27,27,83,87,87
2,38,36,20,77,80,80
3,66,37,37,83,83,83
4,71,34,38,83,87,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1125
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1315, 1394, 1634, 1505, 1465, 1439, 1607, 1535, 1453, 1659]
Pseudo-random, avergae swaps needed: 1472.37573

Q2
Mean Words: 1050
Algo score, swaps needed: 944
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1421, 1382, 1455, 1456, 1479, 1394, 1424, 1413, 1402, 1428]
Pseudo-random, avergae swaps needed: 1449.94569

Q3
Mean Words: 839
Algo score, swaps needed: 843
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1227, 1058, 1426, 1586, 1506, 1602, 1479, 1293, 1737, 1374]
Pseudo-random, avergae swaps needed: 1458.73772

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,93,90
1,24.0,25.0,87,87
2,26.5,24.0,77,80
3,23.5,23.0,90,87
4,26.5,21.0,83,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1751
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2033, 1915, 2086, 2276, 2126, 2160, 2283, 1980, 2007, 2017]
Pseudo-random, avergae swaps needed: 2045.81544

Q2
Mean Words: 2137
Algo score, swaps needed: 1716
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1870, 1833, 2224, 1909, 1795, 1951, 2018, 2188, 1986, 2039]
Pseudo-random, avergae swaps needed: 1994.36782

Swaps (Machine):
[1003, 1199, 907, 1293, 1274, 87, 932, 839, 1125, 944, 843, 1751, 1716]
N: 13 	Mean:  1070.2307692307693 	Var:  163788.02366863904
Avergae swaps (Pseudo-random):
[1499.83852, 1566.62592, 1560.11682, 1837.725, 1757.04872, 182.39354, 1324.63885, 1219.81824, 1472.37573, 1449.94569, 1458.73772, 2045.81544, 1994.36782]
N: 13 	Mean:  1489.957539230769 	Var:  198423.03916664407 

1 ) 1499.83852 > 1003 => 39.702003627465345
2 ) 1566.62592 > 1199 => 26.585368421771225
3 ) 1560.11682 > 907 => 52.94575552364804
4 ) 1837.725

In [17]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,83,80,83
1,10,30,32,83,80,90
2,12,39,24,90,83,83
3,15,38,16,87,83,83
4,13,43,19,87,83,80


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1032
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1439, 1373, 1372, 1540, 1467, 1440, 1525, 1604, 1728, 1337]
Pseudo-random, avergae swaps needed: 1499.07619

Q1
Mean Words: 2048
Algo score, swaps needed: 1217
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1524, 1576, 1468, 1618, 1483, 1448, 1659, 1571, 1490, 1578]
Pseudo-random, avergae swaps needed: 1566.82014

Q2
Mean Words: 947
Algo score, swaps needed: 867
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1508, 1641, 1393, 1717, 1448, 1482, 1581, 1546, 1726, 1788]
Pseudo-random, avergae swaps needed: 1560.14834

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,87
1,20.0,18.0,73,80
2,24.0,25.0,90,80
3,18.0,22.0,77,87
4,17.0,17.0,77,80


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1230
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1815, 1765, 1860, 1982, 1626, 1820, 1950, 1963, 2007, 1930]
Pseudo-random, avergae swaps needed: 1837.55179

Q2
Mean Words: 1546
Algo score, swaps needed: 1202
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1855, 1801, 1800, 1659, 1872, 1771, 1712, 1844, 1776, 1749]
Pseudo-random, avergae swaps needed: 1757.29276

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 1016
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1296, 1308, 1385, 1458, 1196, 1340, 1132, 1158, 1489, 1414]
Pseudo-random, avergae swaps needed: 1324.36144

Q2
Mean Words: 570
Algo score, swaps needed: 1001
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1328, 1258, 1297, 807, 1290, 1134, 1336, 1139, 1231, 1146]
Pseudo-random, avergae swaps needed: 1219.36599

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,83,83,80
1,53,27,27,80,83,87
2,38,36,20,80,77,77
3,66,37,37,83,80,83
4,71,34,38,83,87,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1066
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1175, 1551, 1379, 1387, 1471, 1572, 1533, 1373, 1408, 1512]
Pseudo-random, avergae swaps needed: 1472.61349

Q2
Mean Words: 1050
Algo score, swaps needed: 1017
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1429, 1425, 1351, 1400, 1423, 1469, 1346, 1547, 1654, 1380]
Pseudo-random, avergae swaps needed: 1450.17734

Q3
Mean Words: 839
Algo score, swaps needed: 758
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1492, 1496, 1344, 1206, 1516, 1484, 1322, 1367, 1445, 1551]
Pseudo-random, avergae swaps needed: 1458.80484

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,90
1,24.0,25.0,90,87
2,26.5,24.0,80,80
3,23.5,23.0,87,90
4,26.5,21.0,87,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1769
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2100, 2148, 1912, 2089, 1972, 1874, 2187, 1919, 2043, 2109]
Pseudo-random, avergae swaps needed: 2045.43644

Q2
Mean Words: 2137
Algo score, swaps needed: 1696
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1978, 1914, 1946, 1809, 2199, 2156, 2013, 1841, 2096, 2357]
Pseudo-random, avergae swaps needed: 1993.35733

Swaps (Machine):
[1032, 1217, 867, 1230, 1202, 82, 1016, 1001, 1066, 1017, 758, 1769, 1696]
N: 13 	Mean:  1073.3076923076924 	Var:  159354.67455621302
Avergae swaps (Pseudo-random):
[1499.07619, 1566.82014, 1560.14834, 1837.55179, 1757.29276, 182.54019, 1324.36144, 1219.36599, 1472.61349, 1450.17734, 1458.80484, 2045.43644, 1993.35733]
N: 13 	Mean:  1489.8112523076923 	Var:  198308.55939309226 

1 ) 1499.07619 > 1032 => 36.90731964888027
2 ) 1566.82014 > 1217 => 25.132380858484634
3 ) 1560.14834 > 867 => 57.11627333004295
4 ) 183

In [18]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,83,87,83
1,10,30,32,87,83,87
2,12,39,24,87,67,80
3,15,38,16,87,87,87
4,13,43,19,80,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1300
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1446, 1415, 1786, 1564, 1636, 1455, 1402, 1497, 1630, 1414]
Pseudo-random, avergae swaps needed: 1499.67812

Q1
Mean Words: 2048
Algo score, swaps needed: 1250
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1516, 1695, 1538, 1685, 1799, 1561, 1579, 1628, 1505, 1497]
Pseudo-random, avergae swaps needed: 1566.3041

Q2
Mean Words: 947
Algo score, swaps needed: 1170
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1630, 1553, 1802, 1449, 1441, 1656, 1506, 1624, 1564, 1734]
Pseudo-random, avergae swaps needed: 1559.45462

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,87,87
1,20.0,18.0,77,83
2,24.0,25.0,90,87
3,18.0,22.0,83,77
4,17.0,17.0,77,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1606
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1858, 1825, 1850, 1929, 1742, 1703, 1523, 1642, 1814, 1840]
Pseudo-random, avergae swaps needed: 1837.92491

Q2
Mean Words: 1546
Algo score, swaps needed: 1570
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1933, 1755, 1673, 1706, 1823, 1830, 1588, 1760, 1729, 1913]
Pseudo-random, avergae swaps needed: 1757.53228

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 927
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1476, 1361, 1376, 1515, 1398, 1451, 1320, 1301, 1450, 1327]
Pseudo-random, avergae swaps needed: 1324.2234

Q2
Mean Words: 570
Algo score, swaps needed: 1013
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1276, 1258, 1063, 1189, 1334, 1267, 1342, 1101, 1159, 1122]
Pseudo-random, avergae swaps needed: 1219.16415

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,80,83,83
1,53,27,27,83,87,83
2,38,36,20,87,83,83
3,66,37,37,83,83,83
4,71,34,38,83,87,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1402
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1517, 1456, 1301, 1497, 1536, 1241, 1472, 1315, 1510, 1415]
Pseudo-random, avergae swaps needed: 1472.77932

Q2
Mean Words: 1050
Algo score, swaps needed: 1187
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1331, 1559, 1037, 1545, 1224, 1317, 1306, 1592, 1580, 1454]
Pseudo-random, avergae swaps needed: 1450.30884

Q3
Mean Words: 839
Algo score, swaps needed: 1073
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1449, 1460, 1497, 1483, 1692, 1537, 1277, 1484, 1356, 1486]
Pseudo-random, avergae swaps needed: 1458.28459

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,90
1,24.0,25.0,87,83
2,26.5,24.0,77,83
3,23.5,23.0,87,87
4,26.5,21.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1668
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1802, 1836, 2253, 1731, 2114, 2186, 2277, 2046, 2097, 2222]
Pseudo-random, avergae swaps needed: 2045.44964

Q2
Mean Words: 2137
Algo score, swaps needed: 1746
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2004, 2187, 2302, 2156, 1978, 2338, 1994, 2036, 1905, 2055]
Pseudo-random, avergae swaps needed: 1994.15405

Swaps (Machine):
[1300, 1250, 1170, 1606, 1570, 114, 927, 1013, 1402, 1187, 1073, 1668, 1746]
N: 13 	Mean:  1232.7692307692307 	Var:  166485.5621301775
Avergae swaps (Pseudo-random):
[1499.67812, 1566.3041, 1559.45462, 1837.92491, 1757.53228, 182.5214, 1324.2234, 1219.16415, 1472.77932, 1450.30884, 1458.28459, 2045.44964, 1994.15405]
N: 13 	Mean:  1489.829186153846 	Var:  198405.57343147194 

1 ) 1499.67812 > 1300 => 14.264362647517498
2 ) 1566.3041 > 1250 => 22.462354118647916
3 ) 1559.45462 > 1170 => 28.537174946693195
4 ) 1837

In [19]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,80,87,83
1,10,30,32,87,87,87
2,12,39,24,87,67,77
3,15,38,16,87,87,83
4,13,43,19,83,83,80


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1259
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1429, 1774, 1519, 1662, 1774, 1288, 1339, 1498, 1560, 1663]
Pseudo-random, avergae swaps needed: 1499.68237

Q1
Mean Words: 2048
Algo score, swaps needed: 1183
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1776, 1484, 1591, 1260, 1514, 1586, 1685, 1660, 1441, 1561]
Pseudo-random, avergae swaps needed: 1566.82689

Q2
Mean Words: 947
Algo score, swaps needed: 1220
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1678, 1583, 1733, 1578, 1506, 1326, 1717, 1629, 1531, 1580]
Pseudo-random, avergae swaps needed: 1559.76553

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,90,87
1,20.0,18.0,80,77
2,24.0,25.0,90,83
3,18.0,22.0,87,73
4,17.0,17.0,77,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1558
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1876, 1658, 2032, 1886, 1892, 1641, 1718, 1779, 1566, 1928]
Pseudo-random, avergae swaps needed: 1837.55757

Q2
Mean Words: 1546
Algo score, swaps needed: 1489
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1929, 1780, 1871, 1689, 1703, 1816, 1796, 1772, 1729, 1879]
Pseudo-random, avergae swaps needed: 1757.11228

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 824
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1399, 1318, 1356, 1195, 1401, 1233, 1569, 1365, 1360, 1088]
Pseudo-random, avergae swaps needed: 1324.10488

Q2
Mean Words: 570
Algo score, swaps needed: 920
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1320, 1251, 1133, 1389, 1170, 1310, 1054, 1224, 1337, 1158]
Pseudo-random, avergae swaps needed: 1219.32686

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,80,83,87
1,53,27,27,83,87,83
2,38,36,20,87,83,83
3,66,37,37,83,87,83
4,71,34,38,87,90,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1325
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1461, 1571, 1422, 1513, 1453, 1581, 1474, 1693, 1455, 1576]
Pseudo-random, avergae swaps needed: 1472.36276

Q2
Mean Words: 1050
Algo score, swaps needed: 1179
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1364, 1406, 1576, 1552, 1436, 1203, 1543, 1340, 1198, 1654]
Pseudo-random, avergae swaps needed: 1449.55954

Q3
Mean Words: 839
Algo score, swaps needed: 1102
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1147, 1470, 1378, 1596, 1420, 1578, 1444, 1539, 1473, 1282]
Pseudo-random, avergae swaps needed: 1457.63258

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,93,90
1,24.0,25.0,90,83
2,26.5,24.0,73,83
3,23.5,23.0,87,87
4,26.5,21.0,90,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1768
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1925, 1928, 1881, 1794, 2081, 2026, 2152, 2170, 1965, 2389]
Pseudo-random, avergae swaps needed: 2044.68531

Q2
Mean Words: 2137
Algo score, swaps needed: 1714
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1942, 1966, 2004, 2111, 2044, 1900, 1832, 2073, 2258, 2005]
Pseudo-random, avergae swaps needed: 1994.37063

Swaps (Machine):
[1259, 1183, 1220, 1558, 1489, 81, 824, 920, 1325, 1179, 1102, 1768, 1714]
N: 13 	Mean:  1201.6923076923076 	Var:  176941.90532544377
Avergae swaps (Pseudo-random):
[1499.68237, 1566.82689, 1559.76553, 1837.55757, 1757.11228, 182.43899, 1324.10488, 1219.32686, 1472.36276, 1449.55954, 1457.63258, 2044.68531, 1994.37063]
N: 13 	Mean:  1489.6481684615387 	Var:  198351.35282736438 

1 ) 1499.68237 > 1259 => 17.44908167880161
2 ) 1566.82689 > 1183 => 27.916440223624406
3 ) 1559.76553 > 1220 => 24.44562509558135
4 ) 18

[back to contents](#Contents)